In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def plot(x, y, x_label=None, y_label=None, title=None, name_save_file=None, points=None):
    if title:
        plt.title(title)
    if x_label:
        plt.xlabel(x_label)
    if y_label:
        plt.ylabel(y_label)
    if points:
        for point in points:
            plt.scatter(*point, c="green")
    plt.plot(x, y)
    plt.grid()
    if name_save_file:
        plt.savefig(name_save_file)
    plt.show()

def calc_mutual_inductance(coil_1, coil_2, d, po=0, fi=0):
    mu0 = 4 * np.pi * 10 ** (-7)
    mutual_inductance = np.ones((len(coil_1), len(coil_2)))
    N = 90
    K = 90
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    for ri in range(len(coil_1)):
        for rj in range(len(coil_2)):
            M = 0
            for n in range(N):
                for k in range(K):
                    xk_xn = po + coil_1[ri] * np.cos(df2 * k) * np.cos(fi) - coil_2[rj] * np.cos(df1 * n)
                    yk_yn = coil_1[ri] * np.sin(df2 * k) * np.cos(fi) - coil_2[rj] * np.sin(df1 * n)
                    zk_zn = d + coil_1[ri] * np.cos(df2 * k) * np.sin(fi)
                    r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                    M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
            M *= mu0 * coil_1[ri] * coil_2[rj] / (4 * np.pi)
            mutual_inductance[ri][rj] = M
    return np.sum(mutual_inductance)

def calc_self_inductance(coil, thin=0.001):
    mu0 = 4 * np.pi * 10 ** (-7)
    L = np.sum(mu0 * coil * (np.log(8 * coil / thin) - 7 / 4 + (thin ** 2) / (8 * coil ** 2) * (np.log(8 * coil / thin) + 1 / 3)))
    mutual_inductance = np.ones((len(coil), len(coil)))
    N = 90
    K = 90
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    d = 0
    po = 0
    fi = 0
    for ri in range(len(coil)):
        for rj in range(len(coil)):
            M = 0
            if ri != rj:
                for n in range(N):
                    for k in range(K):
                        xk_xn = po + coil[ri] * np.cos(df2 * k) * np.cos(fi) - coil[rj] * np.cos(df1 * n)
                        yk_yn = coil[ri] * np.sin(df2 * k) * np.cos(fi) - coil[rj] * np.sin(df1 * n)
                        zk_zn = d + coil[ri] * np.cos(df2 * k) * np.sin(fi)
                        r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                        M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
                M *= mu0 * coil[ri] * coil[rj] / (4 * np.pi)
            mutual_inductance[ri][rj] = M
    M = np.sum(mutual_inductance)
    L += M
    return L

def calc_coupling_coefficient(coil_1, coil_2, d, po=0, fi=0):
    M = calc_mutual_inductance(coil_1, coil_2, d, po=po, fi=fi)
    L1 = calc_self_inductance(coil_1)
    L2 = calc_self_inductance(coil_2)
    k = M / (L1 * L2) ** 0.5
    return k

## Next ascent hill climbing в применении к поиску двух витков внутри катушки ##

In [3]:
# размеры витков для первой катушки индуктивности в метрах
coil_1 = np.linspace(0.028, 0.07, 4)

# радиус проволоки
thin = 0.001

start = 0.03 + 2 * thin
finish = 0.09 - 2 * thin
# размеры витков для второй катушки индуктивности в метрах
coil_2 = np.array([0.03, start, finish, 0.09])

# расстояние между катушками в метрах
d = 0.005

In [4]:
def mutation(start, finish, x, r):
    x = np.random.uniform(low = start if x - r < start else x - r,
                          high = finish if x + r > finish else x + r)
    return x


# между витками одной катушки не должно быть пересечений
def test_coils(coil, thin):
    # между витками 2 и 3 катушки есть пересечение
    if coil[1] > coil[2]:
        if (coil[1] - coil[2] - 2 * thin) < 0:
            return False
    elif coil[2] > coil[1]:
        if (coil[2] - coil[1] - 2 * thin) < 0:
            return False
    return True

good_mutation = []
bad_mutation = []
all_mutation = []

Q = 2
band = 0.01
thr_k = 1e-9

In [5]:
coil_2[1] = mutation(start, finish, coil_2[1], band)
coil_2[2] = mutation(start, finish, coil_2[2], band)
print("Начальное значение катушки:", coil_2)

# while True:
for i in range(100):
    fit_k = calc_coupling_coefficient(coil_1, coil_2, d)
    flag = False
    for q in range(Q):
        coil_2q = coil_2.copy()
        coil_2q[1 + q] = mutation(start, finish, coil_2q[1 + q], band)
        
        # проверка на пересечение витков внутри катушки
        while not test_coils(coil_2q, thin):
            print(f"Есть пересечения внутри катушки! {coil_2q}")
            coil_2q[1 + q] = mutation(start, finish, coil_2q[1 + q], band)
        
        fit_kq = calc_coupling_coefficient(coil_1, coil_2q, d)
        
        if fit_kq > fit_k:
            coil_2 = coil_2q.copy()
            fit_k = calc_coupling_coefficient(coil_1, coil_2, d)
            flag = True
            good_mutation.append((fit_kq, coil_2q))
        else:
            bad_mutation.append((fit_kq, coil_2q))
        all_mutation.append((fit_kq, coil_2q))
    print(f"iter: {i} k = {fit_k}, катушка = {coil_2}")

Начальное значение катушки: [0.03       0.03801924 0.0784007  0.09      ]
iter: 0 k = 0.5683094914019051, катушка = [0.03       0.04083546 0.0784007  0.09      ]
iter: 1 k = 0.5683094914019051, катушка = [0.03       0.04083546 0.0784007  0.09      ]
iter: 2 k = 0.5833981392487215, катушка = [0.03       0.04910753 0.0784007  0.09      ]
iter: 3 k = 0.6309801717205947, катушка = [0.03       0.04910753 0.07239985 0.09      ]
iter: 4 k = 0.6485409350663557, катушка = [0.03       0.05003496 0.06626403 0.09      ]
iter: 5 k = 0.6489824011897346, катушка = [0.03       0.05003496 0.06737216 0.09      ]
iter: 6 k = 0.6489824011897346, катушка = [0.03       0.05003496 0.06737216 0.09      ]
iter: 7 k = 0.6489824011897346, катушка = [0.03       0.05003496 0.06737216 0.09      ]
iter: 8 k = 0.6489824011897346, катушка = [0.03       0.05003496 0.06737216 0.09      ]
iter: 9 k = 0.6489824011897346, катушка = [0.03       0.05003496 0.06737216 0.09      ]
iter: 10 k = 0.6543436968686657, катушка = [0.

iter: 92 k = 0.6551462443090599, катушка = [0.03       0.05504249 0.0680738  0.09      ]
iter: 93 k = 0.6551462443090599, катушка = [0.03       0.05504249 0.0680738  0.09      ]
iter: 94 k = 0.6551462443090599, катушка = [0.03       0.05504249 0.0680738  0.09      ]
iter: 95 k = 0.6551462443090599, катушка = [0.03       0.05504249 0.0680738  0.09      ]
iter: 96 k = 0.6551462443090599, катушка = [0.03       0.05504249 0.0680738  0.09      ]
iter: 97 k = 0.6551462443090599, катушка = [0.03       0.05504249 0.0680738  0.09      ]
iter: 98 k = 0.6551462443090599, катушка = [0.03       0.05504249 0.0680738  0.09      ]
iter: 99 k = 0.6551462443090599, катушка = [0.03       0.05504249 0.0680738  0.09      ]


In [6]:
print(f"kmax = {fit_k}; R2max = {coil_2} м")
print(f"Количество полезных мутаций: {len(good_mutation)}")
print(f"Количество вредных мутаций: {len(bad_mutation)}")
print(f"Всего мутаций: {len(all_mutation)}")

kmax = 0.6551462443090599; R2max = [0.03       0.05504249 0.0680738  0.09      ] м
Количество полезных мутаций: 10
Количество вредных мутаций: 190
Всего мутаций: 200
